This notebook uses the MCMC samples for $\boldsymbol{\alpha}$ and $\boldsymbol{\lambda}$, to compute the full-bayesian results.

- The function full_bayes_estimator takes a dataframe "YY" of shape $n\times M$ with $n$ the number of observations and $M$ the size of the sample of $(\boldsymbol{\Lambda}_k)_{k=1}^M,$ and ratio_is is a list of $n$ dataframes $M\times N$, where each column of the dataframes is the importance sampling ratios $\frac{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{A_i}})}{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{\alpha}^\star})}$, where $\boldsymbol{A_i}$ is a MCMC sample for $\boldsymbol{\alpha}.$ It returns the full-bayesian estimator $$e_j = \frac{1}{N}\sum_{i=1}^N \frac{\sum_{k=1}^M YY_{j,k} \frac{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{A_i}})}{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{\alpha}^\star})}}{\sum_{k=1}^M  \frac{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{A_i}})}{p_{\boldsymbol{\Lambda}}(\boldsymbol{\Lambda}_k \mid \boldsymbol{\boldsymbol{\alpha}^\star})}}$$ 

- The function plot_transpo_fullbayes computes the prediction mean and standard deviation, from the "dic_Lambda" the list of samples of $\boldsymbol{\Lambda}$ obtained for each idx_loo, "dic_alpha" the list of samples of $\boldsymbol{\alpha}$ for each idx_loo, "Ysimu_list" and "Ystd_list" the GP means and standard deviations, and "alpha_df" the $\boldsymbol{\alpha}_{\text{MAP}}$ obtained for all idx_loo. 

- The function compute_error_fullbayes computes the RMSRE and the levels of the prediction intervals from ratio_is (same argument as full_bayes_estimator) and Ysimu_list and Ystd_list the GP means and standard deviations.

- The function full_bayes_results uses for each observation variable, plot_transpo_fullbayes and compute_error_fullbayes to compute the performance metrics

In [5]:
def full_bayes_estimator(YY, ratio_is):
    res = []
    for ii in range(10): #for each observation point $x_j$
        l1 = []
        for kk in range(ratio_is[ii].shape[1]): #for each alpha sample
            l1.append(np.average(YY.iloc[ii,:], weights = ratio_is[ii][:,kk])) #get the weighted mean, where the weights are the importance sampling ratios
        res.append(np.mean(l1)) #get the overall mean over 
    return np.array(res)
    

def plot_transpo_fullbayes(dic_Lambda, dic_alpha, Ysimu_list, Ystd_list, alpha_df):
    predictions = pd.DataFrame(np.zeros((10, 3)))
    predictions_square = pd.DataFrame(np.zeros((10, 3)))
    denom_ratio_is = [p_lambda_df(df_Lambda = dic_Lambda[ii], alpha = alpha_df[ii], index_lambda_p=index_lambda_p, index_lambda_q = index_lambda_q, scale = scale) for ii in range(10)] #compute the denominator of the importance sampling ratios
    ratio_is = [np.concatenate([p_lambda_df(df_Lambda = dic_Lambda[ii], alpha = alpha_prov, index_lambda_p=index_lambda_p, index_lambda_q = index_lambda_q, scale = scale).values.reshape(-1,1) for alpha_prov in dic_alpha[ii]], axis=1)/denom_ratio_is[ii].values.reshape(-1,1) for ii in range(10)] #compute the importance sampling ratios
    for index_predict in range(1,4):
        YY = pd.concat([pd.DataFrame(Ysimu_list[ii].iloc[:, index_predict-1]) for ii in range(len(Ysimu_list))], axis=1) # get all the output values (dim M x 10)
        std_mm = pd.concat([pd.DataFrame(Ystd_list[ii].iloc[:, index_predict-1]) for ii in range(len(Ystd_list))], axis=1) # get all the stds values (dim M x 10)
        pred = full_bayes_estimator(YY, ratio_is) #compute the prediction mean
        pred_square = full_bayes_estimator(YY**2, ratio_is) 
        pred_square = pred_square + full_bayes_estimator(std_mm**2, ratio_is)   # equation 19 line 2 part 1
        predictions.iloc[:, index_predict-1] = pred # result agregation
        predictions_square.iloc[:, index_predict-1] = pred_square # result agregation
    
    std_pred = np.sqrt(predictions_square - predictions**2)  #compute the prediction standard deviation
        
    return pd.concat([predictions, std_pred]), ratio_is

def compute_error_fullbayes(ratio_is, Ysimu_list, Ystd_list):
    predictions = pd.DataFrame(np.zeros((10, 3))) 
    predictions_square = pd.DataFrame(np.zeros((10, 3)))
    res = []
    res_intervals = []
    for index_predict in range(1,4): #for each output
        YY = pd.concat([pd.DataFrame(Ysimu_list[ii].iloc[:, index_predict-1]) for ii in range(len(Ysimu_list))], axis=1) # get all the output values (dim M x 10)
        std_mm = pd.concat([pd.DataFrame(Ystd_list[ii].iloc[:, index_predict-1]) for ii in range(len(Ystd_list))], axis=1) # get all the stds values (dim M x 10)
        pred = full_bayes_estimator(YY, ratio_is) #compute the prediction mean
        eta = abs(pred-true_values[f"Y{index_predict}"]).values
        error = dist2(pred,true_values[f"Y{index_predict}"])
        res.append(error)
        diff_cdf = pd.DataFrame(norm.cdf(((pred+eta).reshape(-1,1) - YY)/std_mm) - norm.cdf(((pred-eta).reshape(-1,1) - YY)/std_mm))    # In of Equation 22
        intervals = full_bayes_estimator(diff_cdf, ratio_is) #compute the levels of the predictions intervals : full equation 22
        res_intervals.append(intervals)
    return res, np.transpose(np.array(res_intervals))


def full_bayes_results(index_calib):
    alpha_df = pd.read_csv(pre_path + f"/calib_{index_calib}/alpha_df.csv", index_col=0).values #get alpha_map for each idx_loo
    dic_alpha = [pd.read_csv(pre_path + f"/calib_{index_calib}/samples_alpha_post_{ii}.csv", index_col=0).values for ii in range(10)] #get the samples alpha for each idx_loo
    dic_Lambda = [pd.read_csv(pre_path + f"/calib_{index_calib}/lambd_post_{ii}.csv", index_col=0)for ii in range(10)]#get the samples lambda for each idx_loo
    YY = pd.read_csv(pre_path + f"/calib_{index_calib}/Ysimu_keep.csv", index_col = 0) #get the output values
    Ysimu_keep = np.array_split(YY, len(YY) // 10)# Crée une liste de DataFrames
    Ystd = pd.read_csv(pre_path + f"/calib_{index_calib}/Ystd_keep.csv", index_col = 0) #get the stds values
    Ystd_keep = np.array_split(Ystd, len(Ystd) // 10)

    plot1 = plot_transpo_fullbayes(dic_Lambda = dic_Lambda, dic_alpha = dic_alpha, Ysimu_list = Ysimu_keep, Ystd_list = Ystd_keep, alpha_df = alpha_df) #compute prediction mean and std
    save_results(plot1[0], "full_bayes.csv", pre_path = pre_path, calib = index_calib) 
    ratio_is = plot1[1]

    errors_bayes, intervals_bayes = compute_error_fullbayes(ratio_is, Ysimu_keep, Ystd_keep) #compute RMSRE and levels of prediction intervals
    save_results(pd.DataFrame(errors_bayes), "errors_bayes.csv", pre_path = pre_path, calib = index_calib)
    save_results(pd.DataFrame(intervals_bayes), "intervals_bayes.csv", pre_path = pre_path, calib = index_calib)
    
    
[full_bayes_results(index_calib) for index_calib in calib_only]